In [1]:
import numpy as np
from datascience import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

import numpy as np
from scipy import stats
import subprocess
import pandas as pd
import time
from tqdm import tqdm

import requests
from lxml import etree
from lxml.etree import fromstring

def get_sequence(chrom,start, stop):

    req = requests.get(("http://genome.ucsc.edu/cgi-bin/das/mm10/dna?segment=" + chrom + ":" + str(int(start)) + "," + str(int(stop))), stream=True)
    req.raw.decode_content = True  # ensure transfer encoding is honoured
    b = etree.parse(req.raw)
    return "".join([i[1:] for i in str(etree.tostring(b)).split("length")[1].split("\\n</DNA>\\")[0].split(">")[1].split("\\")[1:]])


In [2]:
reads = Table.read_table("cond2_predictions.csv")#.sort(5, descending=True)
reads

chrom,chromStart,chromEnd,name,strand,differential binding (unitless)
chr12,8.58156e+07,8.58156e+07,0610007P14Rik,-,62.2749
chr12,8.58157e+07,8.58157e+07,0610007P14Rik,-,62.5928
chr12,8.58158e+07,8.58158e+07,0610007P14Rik,-,86.8699
chr12,8.5816e+07,8.5816e+07,0610007P14Rik,-,29.1225
chr11,5.16855e+07,5.16856e+07,0610009B22Rik,-,59.7131
chr11,5.16856e+07,5.16856e+07,0610009B22Rik,-,56.1821
chr18,3.82619e+07,3.8262e+07,0610009O20Rik,+,18.1918
chr2,1.7662e+08,1.7662e+08,0610010B08Rik,-,6.40211
chr2,1.7662e+08,1.7662e+08,0610010B08Rik,-,34.2425
chr2,1.7662e+08,1.76621e+08,0610010B08Rik,-,106.135


In [ ]:
reads = Table.read_table("cond2_predictions.csv").sort(5, descending=True)

sequences = make_array()

for i in tqdm(np.arange(reads.num_rows)):
    chrom = reads.column("chrom").item(i)
    start = reads.column("chromStart").item(i)
    stop = reads.column("chromEnd").item(i)
    sequence = get_sequence(chrom,start, stop)
    sequences = np.append(sequences, sequence)

 32%|███▏      | 18220/56156 [1:24:57<2:04:07,  5.09it/s] 

In [ ]:
sequences = [i.replace("t", "u") for i in sequences]
reads = reads.with_column("RNA sequences", sequences)
reads.with_column("Seed Sequence?", [("ugcauuaa" in i) for i in reads.column("RNA sequences")]).where("Seed Sequence?", True).to_csv("test.csv")#.group("Seed Sequence?")

In [ ]:
chrom = "chr5"
start = 142903034
stop = 142903034 + 10
get_sequence(chrom,start, stop)

In [ ]:
reads.num_rows

In [ ]:
5000 